# Converting the csv files to a SQL database

In [ ]:
import pandas as pd
import sqlite3
import os
import re

In [ ]:
%load_ext autoreload
%autoreload 2

# my code
from SQLutils import addCSVToSQLdb, createNewSQLdbFromCSVs

## Create new database files with the desired paragraph(s)

In [ ]:
allPara = pd.read_csv('../csv/paragraphs.csv')
allParaName = allPara['paragraphname'].to_list()
allParaName

In [ ]:
#all that we currently have
createNewSQLdbFromCSVs('Dec2021_all', allParaName)

In [ ]:
#default, available on load
default = ['Polymer Composite Example','Aluminum-CHiMaDExample','Thermoelectrics_test','Emery-Exam1-M54']
createNewSQLdbFromCSVs('default', default)

In [ ]:
createNewSQLdbFromCSVs('testing', default)

In [ ]:
useCases = ['Polymer Composite Example','Aluminum-CHiMaDExample','Thermoelectrics_test','Emery-Exam1-M54',
    'CJE BCPs for DSA',
    'Co_alloy_WhitneyT',
    'Cobalt-Based Superalloys_Brandonfinished',
    'Metal AM - SLM',
    'PECs',
    'PECs_class',
    'Superalloy -test',
    'Thermoelectrics_JM',
    'whitney DSA',
    'CJE DSA Random BCPs'
]
createNewSQLdbFromCSVs('CHiMaD_UseCases', useCases)

# Create the metrics database

In [ ]:
dbName = 'CHiMaD_metrics.db'
tableName = 'loginMetrics'
columns = ['timestamp', 'name', 'organization', 'email', 'purpose', 'groupname']

In [ ]:
# connect to the SQL database (this will create a file if it does not already exist)
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

# include all columns, and assume that they are all text
# allow only alphanumeric values in column names
cols = ''
for col in columns:
    cc = re.sub(r'\W+', '', col) 
    cols += cc + ' text, ' 
cols = cols[:-2]
    
# create the table
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

cursor.close()

# Remove a table from a file

In [ ]:
dbName = 'default.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

dropTableStatement = "DROP TABLE dbs"
cursor.execute(dropTableStatement)
cursor.close()

# Remove a row from a database file

In [ ]:
#read in table 
dbName = 'default.db'
tablename = 'polymercompositeexample'

conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]
print(tables)

cursor.execute('SELECT * FROM ' + tablename)
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns)  
display(df)

# perform selection
#df = df.loc[~((df['username'] == 'AaronGeller') & (df.index > 41))]
# rewrite table
#df.to_sql(tablename, conn, if_exists='replace', index = False)
#display(df)

cursor.close()

## Test selecting everything from the database and puting back into DataFrame

In [ ]:
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]

for t in tables:
    print(t)
    cursor.execute('SELECT * FROM ' + t)
    columns = [description[0] for description in cursor.description]
    df2 = pd.DataFrame(cursor.fetchall(), columns = columns)    
    display(df2)


In [ ]:
cursor.close()

# Manual testing

## Read in a csv file

In [ ]:
#csvName = 'thermoelectricsjm.csv' # this has a column with ".1" in the name, and sqlite does not allow it
csvName = 'paragraphs.csv' # this has a column with ".1" in the name, and sqlite does not allow it
df = pd.read_csv('../csv/' + csvName)
df

## Create a SQL database

In [ ]:
dbName = 'CHiMaD_SDC.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

In [ ]:
# the database name will be the file name w/o csv
# take only alphanumeric values
tableName = re.sub(r'\W+', '',csvName.replace('.csv','')) 
tableName

In [ ]:
# include all columns, and assume that they are all text
cols = ''
for col in df.columns:
    cc = re.sub(r'\W+', '', col) # only alphanumeric values in column names
    cols += cc + ' text, '
cols = cols[:-2]
cols

In [ ]:
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

In [ ]:
# add the dataframe into the database
df.to_sql(tableName, conn, if_exists='replace', index = False)

In [ ]:
#grab one table and write to file
t = 'paragraphs'
cursor.execute('SELECT * FROM ' + t)
columns = [description[0] for description in cursor.description]
df2 = pd.DataFrame(cursor.fetchall(), columns = columns)   
df2.to_csv(t+'.csv', index=False)

In [ ]:
#read in table to check
# dbName = 'CHiMaD_metrics.db'
# tablename = 'loginMetrics'
#dbName = 'uchicago.db'
#dbName = 'dec2021_all.db'
#tablename = 'paragraphs'
#tablename = 'polymercompositeexample'
# dbName = 'available_dbs.db'
# tablename = 'dbs'
dbName = 'default.db'
tablename = 'polymercompositeexample'
#tablename = 'paragraphs'

conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]
print(tables)

cursor.execute('SELECT * FROM ' + tablename)
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns)  
display(df)

cursor.close()

In [ ]:
display(df)
df.replace({'groupname': {'testing': 'foo'}}, inplace = True)
display(df)

In [ ]:
#rename table
dbName = 'testing1.db'
tablename = 'polymercompositeexample'
newname = 'foo'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("ALTER TABLE `" + newname + "` RENAME TO `" + tablename + "`")
cursor.close()


In [ ]:
#rename value
paragraphname = 'Polymer Composite Example'
newname = 'foo'
db = 'testing1.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()
cursor.execute('SELECT * FROM paragraphs')
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns) 
df.replace({'paragraphname': {paragraphname: newname}}, inplace = True)
df.to_sql('paragraphs', conn, if_exists='replace', index = False)
cursor.close()

In [ ]:
cursor.close()

In [ ]:
# command to copy files from Downloads and change names
#find ~/Downloads/ | grep testParagraph | awk '{print ("cp "$0" "$3)}' | sed s/" - "/"\\\ -\\\ "/ > foo